## Review response generation (demo)

This is a demo script for automatic response generation models trained with Fairseq.

In [1]:
import sys
import os
import random
# set device
# os.environ["CUDA_VISIBLE_DEVICES"]="6"  # specify which GPU(s) to be used

from typing import List, Optional

import torch
from fairseq import bleu, checkpoint_utils, data, options, tasks, utils
from fairseq.data import encoders
from fairseq.logging.meters import StopwatchMeter


# from fairseq.models.rrgen_seq2seq_lstm import rrgen_lstm_arch

## Define arguments and load model for generation

In [2]:
# Parse command-line arguments for generation
parser = options.get_generation_parser(default_task='rrgen_translation')

# test (small) model
# input_args = [
#     '/srv/scratch2/kew/fairseq_materials/translation_format/raw',
#     '--path=/srv/scratch2/kew/fairseq_materials/translation_format/rrgen_lstm_emb_senti_cate_rate_100k/checkpoints/checkpoint_best.pt',
#     '-s=src',
#     '-t=tgt',
#     '--task=rrgen_translation',
#     '--dataset-impl=raw',
#     '--nbest=1',
#     '--beam=10',
# #     '--sampling',
# #     '--sampling-topp=0.9',
#     '--use-sentiment=senti',
#     '--use-category=cate',
#     '--use-rating=rate',
# ]

input_args = [
    '/srv/scratch2/kew/fairseq_materials/rrgen/de/data_bin_rg',
    '--path=/srv/scratch2/kew/fairseq_materials/rrgen/de/ft100src_rg/checkpoints/checkpoint_best.pt',
    '-s=review',
    '-t=response_rg',
    '--task=rrgen_translation',
    '--dataset-impl=mmap',
    '--nbest=1',
    '--beam=5',
#     '--sampling',
#     '--sampling-topk=10',
#     '--sampling-topp=0.98',
    '--use-sentiment=sentiment',
    '--use-category=domain',
    '--use-rating=rating',
]

args = options.parse_args_and_arch(parser, input_args=input_args)

# Set device
# use_cuda = torch.cuda.is_available() and not args.cpu
use_cuda = False

# Setup task
task = tasks.setup_task(args)

# Load model
print('loading model from {}'.format(args.path))
models, _model_args = checkpoint_utils.load_model_ensemble(
    [args.path], task=task)
model = models[0]

print('finished loading model from {}'.format(args.path))

if use_cuda:
    model.cuda()
    
# Load alignment dictionary for unknown word replacement
# (None if no unknown word replacement, empty if no path to align dictionary)
align_dict = utils.load_align_dict(args.replace_unk)

# initialise generator model
generator = task.build_generator(models, args)

# Handle tokenization and BPE
tokenizer = encoders.build_tokenizer(args)
bpe = encoders.build_bpe(args)

def decode_fn(x):
    if bpe is not None:
        x = bpe.decode(x)
    if tokenizer is not None:
        x = tokenizer.decode(x)
    return x


loading model from /srv/scratch2/kew/fairseq_materials/rrgen/de/ft100src_rg/checkpoints/checkpoint_best.pt
finished loading model from /srv/scratch2/kew/fairseq_materials/rrgen/de/ft100src_rg/checkpoints/checkpoint_best.pt


In [3]:
def get_response(sentence: str,
                 ext_senti: Optional[int] = None,
                 ext_cate: Optional[str] = None,
                 ext_rate: Optional[int] = None,
                 target: Optional[str] = None,
                ):

    # vectorize input sentence 
    tokens = task.source_dictionary.encode_line(
    sentence, add_if_not_exist=False,)

    if args.use_sentiment:
        try:
            ext_senti = task.ext_senti_dict[ext_senti]
        except:
            try:
                ext_senti = task.ext_senti_dict[int(ext_senti)]
            except:
                r = random.choice(list(task.ext_senti_dict))
                print(f'[!] WARNING: Could not find value for sentiment input {ext_senti}. Using sentiment value {r}')               
                ext_senti = task.ext_senti_dict[r]
    if args.use_category:
        try:
            ext_cate = task.ext_cate_dict[ext_cate]
        except:
            try:
                ext_cate = task.ext_cate_dict[int(ext_cate)]
            except:
                r = random.choice(list(task.ext_cate_dict))
                print(f'[!] WARNING: Could not find value for category input {ext_cate}. Using category value {r}')               
                ext_cate = task.ext_cate_dict[r]
    if args.use_rating:
        try:
            ext_rate = task.ext_rate_dict[ext_rate]
        except:
            try:
                ext_rate = task.ext_rate_dict[int(ext_rate)]
            except:
                r = random.choice(list(task.ext_rate_dict))
                print(f'[!] WARNING: Could not find value for category input {ext_rate}. Using rating value {r}')               
                ext_rate = task.ext_rate_dict[r]
                
    # collate input as batch of size 1
    batch = data.rrgen_dataset.collate(
    samples=[{
        'id': -1,
        'source': tokens,
        'ext_senti': ext_senti,
        'ext_rate': ext_rate,
        'ext_cate': ext_cate
    }],
    pad_idx=task.source_dictionary.pad(),
    eos_idx=task.source_dictionary.eos(),
    left_pad_source=False,
    input_feeding=False)

    # [!] ensure correct dtype (int64)
    batch['net_input']['src_tokens'] = batch['net_input']['src_tokens'].type(
        torch.LongTensor)

    batch = utils.move_to_cuda(batch) if use_cuda else batch
    
    # assume no prefix tokens to initialise decoded output
    prefix_tokens = None
    
    gen_timer = StopwatchMeter()
    gen_timer.start()
    # target_tokens = None
    hypos = task.inference_step(generator, model, batch, prefix_tokens)
    num_generated_tokens = sum(len(h[0]['tokens']) for h in hypos)
    gen_timer.stop(num_generated_tokens)
    
    device = 'GPU' if use_cuda else 'CPU'
    
    print('Generated {} sentences ({} tokens) in {:.1f}s on {}\n'.format(
        batch['nsentences'], gen_timer.n, gen_timer.sum, device))
    
    # Process top predictions
    for j, hypo in enumerate(hypos[0][:args.nbest]):
        hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
            hypo_tokens=hypo['tokens'].int().cpu(),
            src_str=tokens,
            alignment=hypo['alignment'],
            align_dict=align_dict,
            tgt_dict=task.target_dictionary,
            remove_bpe=args.remove_bpe,
            # extra_symbols_to_ignore=get_symbols_to_strip_from_output(generator),
        )
        detok_hypo_str = decode_fn(hypo_str)
        
        print(detok_hypo_str)
        print()
        
        if target:
            # Generate and compute BLEU score
            if args.sacrebleu:
                scorer = bleu.SacrebleuScorer()
            else:
                scorer = bleu.Scorer(
                    task.target_dictionary.pad(),
                    task.target_dictionary.eos(),
                    task.target_dictionary.unk())
            
#             if align_dict is not None or args.remove_bpe is not None:
                # Convert back to tokens for evaluation with unk replacement and/or without BPE
            target_tokens = task.target_dictionary.encode_line(target, add_if_not_exist=True)
            hypo_tokens = task.target_dictionary.encode_line(detok_hypo_str, add_if_not_exist=True)
            
            if hasattr(scorer, 'add_string'): # i.e. if using SacreBLEU
                scorer.add_string(target, detok_hypo_str)
            else:
                scorer.add(target_tokens, hypo_tokens) # calculate n-gram overlap on vectorized texts 
            
            print(scorer.result_string())

## Generate review responses with loaded model

In [4]:
review = """
das Allerletzte ---SEP--- wir haben bereits vor Monaten 
zwei Doppelzimmer mit getrennten Betten gebucht , 
die Kreditkarte wurde lange vor der Ankunft belastet , 
obwohl die Zimmer bezahlt sind wird die Kreditkarte bei 
der Ankunft noch einmal mit <DIGIT> Euro Garantie belastet 
( mehr als der Preis der bereits bezahlten Übernachtungen ) , 
bei der Ankunft im Hotel ist die Reception völlig überlaufen , 
Wartezeit eine halbe Stunde , die reservierten und bezahlten 
Zimmer stehen nicht zur Verfügung , das Hotel sei ausgebucht , 
wir bekommen lediglich zwei Zimmer mit kleinen Doppelbetten , 
ein Zimmer nicht sehr sauber , es liegt im Eingangsbereich 
Müll auf dem Boden , das ganze ist eine Zumutung , 
Personal an der Rezeption genervt und unfreundlich , 
dieses Hotel besser meiden , 
P.S. : in der Hotelhalle ist auch noch Erbrochenes auf dem Fußboden
"""
target = """
<GREETING> W , vielen Dank für ihren Aufenthalt und für die Zeit ,
die sie sich genommen haben um uns dieses Feedback zu schreiben .
wir möchten uns natürlich für den negativen Eindruck ,
den sie hier offensichtlich bekommen haben entschuldigen .
da uns die Zufriedenheit unserer Gäste sehr am Herzen liegt ,
möchten wir der Sache gerne auf den Grund gehen und bitten sie daher ,
uns persönlich per E-Mail zu kontaktieren ,
damit wir in diesem Fall recherchieren können .
dies dient zum einen zur internen Verbesserung ,
zum anderen möchten wir ihnen natürlich gerne zeigen ,
dass wir es besser können und eine Lösung für diese Herausforderung finden .
wir freuen uns auf ihre Rückmeldung .
"""
senti = 1
cate = 1
rate = 1

get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate, target=target)

Generated 1 sentences (102 tokens) in 10.3s on CPU

<GREETING> , vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . auch wenn wir es bedauern , dass sie keine besseren Erfahrungen bei uns gemacht haben , hilft uns ihre Beurteilung dazu zu lernen und uns zu verbessern . wir arbeiten daran , es unseren Gästen an nichts fehlen zu lassen . wenn sie uns die Chance geben , ihr Vertrauen zurück zu gewinnen , garantieren wir ihnen , unser Bestes zu geben , damit sie genauso gute Erfahrungen mit uns machen können , wie andere Gäste . <SALUTATION>

BLEU4 = 0.00, 43.6/12.0/1.0/0.0 (BP=0.837, ratio=0.849, syslen=101, reflen=119)


In [5]:
review = """
das war das letzte Mal ---SEP--- unprofessionelle , 
aggressive und gewalttätige Tür ohne jede Klasse und 
ohne die Fähigkeit gewaltfrei zu deeskalieren"""
target = """
<GREETING> , wir bedauern sehr , dass sie so unangenehme
eine Erfahrung bei uns machen Mussten . schreiben sie uns
sehr gerne eine E-Mail an <EMAIL> , damit wir mehr Details
über diesen Abend erfahren und gemeinsam eine Lösung finden können . <SALUTATION>"""
senti = 1
cate = 2
rate = 1
get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate, target=target)

Generated 1 sentences (49 tokens) in 3.7s on CPU

<GREETING> , vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . wir freuen uns , dass ihnen der Aufenthalt bei uns gefallen hat und hoffen , sie bald wieder bei uns begrüßen zu dürfen ! <SALUTATION>

BLEU4 = 0.00, 29.2/8.5/2.2/0.0 (BP=1.000, ratio=1.116, syslen=48, reflen=43)


In [6]:
review = """leider sehr enttäuschend ---SEP--- wir waren 
an einem sonnigen Donnerstagabend auf der Terrasse von The Butcher .
auf unsere Speisekarte mussten wir erst einmal <DIGIT> Minuten warten ,
die Bedienung erklärte uns , dass Zurzeit keine Karten verfügbar seien .
die Bedienung war zwar freundliche , wirkte jedoch gestresst , 
weil alle Tische besetzt waren . 
wir bestellten zwei vegane Burger , da drei der vegetarischen Burger
laut Karte Vegan zubereitet werden könne . die Bedienung erklärte uns dann aber ,
dass dies nur bei einem Burger möglich sei . 
als die Burger nach einer halben Stunde serviert wurden , enthielten sie Käse .
also schickten wir sie zurück in die Küche . 
nach einer Viertelstunde kamen die Burger zurück - diesmal mit einem " V " gekennzeichnet .
sie enthielten noch immer Käse . erst nach insgesamt <DIGIT> Stunden Wartezeit
erhielten wir schliesslich die bestellten veganen Burger , 
dabei handelte es sich aber eher um ein Brötchen mir Gemüse ohne Sauce . ich habe Verständnis dafür , dass nicht in jedem Restaurant Vegan gegessen werden kann . wer dies aber auf die Karte schreibt , sollte doch wenigstens wissen , was Vegan bedeutet . auch irritierend fand ich , dass ein Burger-Restaurant , das auf der Karte damit wirbt , besonders nachhaltig zu sein , alle Getränke im Plastik-Becher serviert . nachhaltig geht anders ."""
senti = 2 # 7
cate = 2
rate = 1

get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate)

Generated 1 sentences (37 tokens) in 4.9s on CPU

<GREETING> , vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . wir hoffen , sie bald wieder bei uns begrüßen zu dürfen ! <SALUTATION>



In [7]:
review = """falsche Zusagen ( schriftlich ) ---SEP--- unsere Anreise erfolgte gegen <DIGIT> Uhr mit <DIGIT> müden Kindern nach einer <DIGIT> stündigen Autofahrt . ich wurde noch begrüßt , mein Mann der <DIGIT> Min später nach kam nicht ( kein Problem ) . vor der Buchung wurde uns schriftlich ein Doppelzimmer mit Verbindungstür bestätigt . vor Ort , wusste keiner etwas davon . Kinderzimmer auf einer Etage , Elternschlafzimmer auf einer anderen ! ? ? ? nach eindringlichem Bitten eine Lösung zu finden , wurde mir gesagt sie seien ausgebucht und das ist eben so ! ? ich habe wehement darum gebeten eine Lösung zu finden - beide angestellte <DIGIT> Min weg ! ? dann wurde mir gesagt , sie versuchen es- kann dauern ..... auf meine Frage , wie lange ( mittlerweile <DIGIT> Uhr ) war die Antwort - keine Ahnung 🤔 . unfassbar wie Unprofessionell . nach eindringlichem Bitten mir eine ungefähre Zeit zu nennen , <DIGIT> Min , <DIGIT> Min oder eine Stunde wurde mir gesagt eher 1stunde . ich habe dankend abgelehnt und darum gebeten den Vorgang zu stornieren . ich hätte dieses Hotel niemals ohne Zusage der Verbindungstür gewählt . vor Ort waren die Mitarbeiter echt frech und ignorant - ich bin schwer enttäuscht . das einzig positive , das sie den Vorgang ohne Kosten storniert haben"""
senti = 1
cate = 1
rate = 1

get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate)

Generated 1 sentences (102 tokens) in 12.3s on CPU

<GREETING> , vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . auch wenn wir es bedauern , dass sie keine besseren Erfahrungen bei uns gemacht haben , hilft uns ihre Beurteilung dazu zu lernen und uns zu verbessern . wir arbeiten daran , es unseren Gästen an nichts fehlen zu lassen . wenn sie uns die Chance geben , ihr Vertrauen zurück zu gewinnen , garantieren wir ihnen , unser Bestes zu geben , damit sie genauso gute Erfahrungen mit uns machen können , wie andere Gäste . <SALUTATION>



In [8]:
review = """Miserabel ---SEP--- im Hotel Seiler ist das Wort Kulanz nicht existent . wir hätten wenigstens ein Entgegenkommen in Form.Z . b.eines Gutscheines erwartet . das Hotel Seiler haben wir aus unserer Liste gestrichen . wir werden auch entsprechend Werbung für das Haus machen ."""
senti = 4
cate = 1
rate = 1
get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate)

Generated 1 sentences (49 tokens) in 4.7s on CPU

<GREETING> , vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . wir freuen uns , dass ihnen der Aufenthalt bei uns gefallen hat und hoffen , sie bald wieder bei uns begrüßen zu dürfen ! <SALUTATION>



In [9]:
review = """stinkend statt dufte ---SEP--- das gesamte Hotel eoch nach Toilette , da es ein paar Wochen zuvor ein Abwasser-Rohrbruch gab . das Badwar ungenügend geputzt und die Klimaanlage ließ sich nicht regeln . das Zimmer war sehr warm . das Personal am Check-In lehnte eine Reklamation ab und wies mehrere Gäste in arroganter und unfreundlicher Weise ab ."""
senti = 3 #7
cate = 1
rate = 1
get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate)

Generated 1 sentences (49 tokens) in 4.7s on CPU

<GREETING> , vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . wir freuen uns , dass ihnen der Aufenthalt bei uns gefallen hat und hoffen , sie bald wieder bei uns begrüßen zu dürfen ! <SALUTATION>



In [10]:
review = """enttäuschend ---SEP--- der Wein ( Pinot Grigio und Frascati ) war absolut nicht zu empfehlen , billige No-Name Ware , das Essen geschmacklos ( Pizza Frutti Di Mare mit Meeresfrüchte-Imitat aus Surimi ) . die Damen im Service freundlich jedoch der Patrone beim Bezahlen der Rechnung unfreundlich und arrogant . nie wieder !"""
senti = 3
cate = 2
rate = 1
get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate)


Generated 1 sentences (101 tokens) in 9.4s on CPU

<GREETING> vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . auch wenn wir es bedauern , dass sie keine besseren Erfahrungen bei uns gemacht haben , hilft uns ihre Beurteilung dazu zu lernen und uns zu verbessern . wir arbeiten daran , es unseren Gästen an nichts fehlen zu lassen . wenn sie uns die Chance geben , ihr Vertrauen zurück zu gewinnen , garantieren wir ihnen , unser Bestes zu geben , damit sie genauso gute Erfahrungen mit uns machen können , wie andere Gäste . <SALUTATION>



In [11]:
review = """es war besser als befriedigend aber nicht sehr gut ---SEP--- Gesamteindruck super , Preise für Garage und Frühstück sind generell in allen Hotels zu hoch . Preis / Nacht wäre besser zu akzeptieren wenn es Fitness/Wellness O.Ä. geben würde . dazu könnten Z.B. Info ´ s zu Partnern ausliegen ."""
senti = 6
cate = 1
rate = 3
get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate)

Generated 1 sentences (49 tokens) in 4.7s on CPU

<GREETING> , vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . wir freuen uns , dass ihnen der Aufenthalt bei uns gefallen hat und hoffen , sie bald wieder bei uns begrüßen zu dürfen ! <SALUTATION>



In [12]:
review = """Weihnachten im Parkhotel lindner Oberstaufen ---SEP--- unvergessliche Weihnachtsatmosphäre im Parkhotel Lindner , zusammen feiern in einer grossen Familie , sehr gediegen mit Harfenmusik , Weihnachtsgeschichten hören und zusammen Weihnachtslieder singen unter einem wunderbar geschmückten Baum , das war Weihnachten <DIGIT> . das freundliche aufgestellte Personal verwöhnte uns Gäste mit auserlesenen Speisen aus der Küche von Herrn Wagenblast . es war einmal mehr ein unvergesslicher Aufenthalt ."""
senti = 8
cate = 1
rate = 5
get_response(sentence=review, ext_senti=senti, ext_cate=cate, ext_rate=rate)

Generated 1 sentences (37 tokens) in 4.7s on CPU

<GREETING> , vielen Dank , dass sie sich die Zeit genommen haben , uns ihre Meinung zu ihrem Aufenthalt bei uns mitzuteilen . wir hoffen , sie bald wieder bei uns begrüßen zu dürfen ! <SALUTATION>

